# 109 persons
## electrodes : 8, 16, 32, 64

In [2]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

2023-10-04 14:39:26.713347: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-04 14:39:26.740469: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-04 14:39:26.740500: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-04 14:39:26.740520: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-04 14:39:26.745637: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-04 14:39:26.746524: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [4]:
no_of_patients=109

In [5]:
train=sorted(glob('files2/*.edf'))
train=train[:no_of_patients]
train

['files2/S001R04.edf',
 'files2/S002R04.edf',
 'files2/S003R04.edf',
 'files2/S004R04.edf',
 'files2/S005R04.edf',
 'files2/S006R04.edf',
 'files2/S007R04.edf',
 'files2/S008R04.edf',
 'files2/S009R04.edf',
 'files2/S010R04.edf',
 'files2/S011R04.edf',
 'files2/S012R04.edf',
 'files2/S013R04.edf',
 'files2/S014R04.edf',
 'files2/S015R04.edf',
 'files2/S016R04.edf',
 'files2/S017R04.edf',
 'files2/S018R04.edf',
 'files2/S019R04.edf',
 'files2/S020R04.edf',
 'files2/S021R04.edf',
 'files2/S022R04.edf',
 'files2/S023R04.edf',
 'files2/S024R04.edf',
 'files2/S025R04.edf',
 'files2/S026R04.edf',
 'files2/S027R04.edf',
 'files2/S028R04.edf',
 'files2/S029R04.edf',
 'files2/S030R04.edf',
 'files2/S031R04.edf',
 'files2/S032R04.edf',
 'files2/S033R04.edf',
 'files2/S034R04.edf',
 'files2/S035R04.edf',
 'files2/S036R04.edf',
 'files2/S037R04.edf',
 'files2/S038R04.edf',
 'files2/S039R04.edf',
 'files2/S040R04.edf',
 'files2/S041R04.edf',
 'files2/S042R04.edf',
 'files2/S043R04.edf',
 'files2/S0

In [6]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [7]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [8]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [9]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [10]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(1308000, 163391, 1308000, 163391)

In [11]:
print(xtest.iloc[:,-1].values)

[-1.0e-05  1.5e-05  6.0e-06 ...  4.0e-06  1.2e-05 -3.0e-06]


In [12]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000077,0.000051,0.000020,-0.000052,-0.000064,0.000143,-0.000131,0.000119,-0.000013,-0.000012,...,0.000050,-0.000005,-0.000011,-0.000044,0.000001,0.000042,0.000003,0.000048,-0.000040,-0.000022
1307996,0.000071,0.000063,0.000092,-0.000048,-0.000063,0.000181,-0.000130,0.000136,-0.000006,0.000016,...,0.000052,0.000007,0.000006,-0.000023,0.000016,0.000049,0.000020,0.000069,-0.000026,-0.000004
1307997,0.000062,0.000065,0.000119,-0.000045,-0.000058,0.000166,-0.000122,0.000145,-0.000007,0.000020,...,0.000042,-0.000002,0.000001,-0.000022,0.000010,0.000039,0.000013,0.000067,-0.000029,-0.000012
1307998,0.000052,0.000061,0.000111,-0.000049,-0.000056,0.000152,-0.000108,0.000123,-0.000017,0.000011,...,0.000038,-0.000011,-0.000006,-0.000029,0.000004,0.000036,0.000008,0.000057,-0.000035,-0.000014


In [13]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [14]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [15]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_28746/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_28746/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_28746/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000077,0.000051,0.000020,-0.000052,-0.000064,0.000143,-0.000131,0.000119,-0.000013,-0.000012,...,0.000050,-0.000005,-0.000011,-0.000044,0.000001,0.000042,0.000003,0.000048,-0.000040,-0.000022
1307996,0.000071,0.000063,0.000092,-0.000048,-0.000063,0.000181,-0.000130,0.000136,-0.000006,0.000016,...,0.000052,0.000007,0.000006,-0.000023,0.000016,0.000049,0.000020,0.000069,-0.000026,-0.000004
1307997,0.000062,0.000065,0.000119,-0.000045,-0.000058,0.000166,-0.000122,0.000145,-0.000007,0.000020,...,0.000042,-0.000002,0.000001,-0.000022,0.000010,0.000039,0.000013,0.000067,-0.000029,-0.000012
1307998,0.000052,0.000061,0.000111,-0.000049,-0.000056,0.000152,-0.000108,0.000123,-0.000017,0.000011,...,0.000038,-0.000011,-0.000006,-0.000029,0.000004,0.000036,0.000008,0.000057,-0.000035,-0.000014


In [16]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [17]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [18]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
40875/40875 [==============================] - 56s 1ms/step - loss: 2.9747 - val_loss: 3.2391
Epoch 2/10
40875/40875 [==============================] - 54s 1ms/step - loss: 2.5099 - val_loss: 3.3174
Epoch 3/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.4206 - val_loss: 3.2931
Epoch 4/10
40875/40875 [==============================] - 60s 1ms/step - loss: 2.3748 - val_loss: 3.3589
Epoch 5/10
40875/40875 [==============================] - 61s 1ms/step - loss: 2.3458 - val_loss: 3.3275
Epoch 6/10
40875/40875 [==============================] - 56s 1ms/step - loss: 2.3254 - val_loss: 3.3775
Epoch 7/10
40875/40875 [==============================] - 54s 1ms/step - loss: 2.3097 - val_loss: 3.4692
Epoch 8/10
40875/40875 [==============================] - 55s 1ms/step - loss: 2.2977 - val_loss: 3.2967
Epoch 9/10
40875/40875 [==============================] - 54s 1ms/step - loss: 2.2875 - val_loss: 3.3769
Epoch 10/10
40875/40875 [==============================

In [19]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

5106/5106 [==============================] - 4s 824us/step
              precision    recall  f1-score   support

           0       0.13      0.13      0.13      1499
           1       0.20      0.20      0.20      1499
           2       0.17      0.20      0.19      1499
           3       0.34      0.38      0.36      1499
           4       0.67      0.64      0.66      1499
           5       0.07      0.04      0.05      1499
           6       0.17      0.47      0.25      1499
           7       0.28      0.28      0.28      1499
           8       0.36      0.19      0.25      1499
           9       0.29      0.42      0.35      1499
          10       0.13      0.12      0.12      1499
          11       0.07      0.08      0.07      1499
          12       0.06      0.00      0.01      1499
          13       0.10      0.08      0.09      1499
          14       0.12      0.03      0.04      1499
          15       0.24      0.18      0.21      1499
          16       0.1

## 0-16

In [20]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [21]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_28746/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_28746/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_28746/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000005,0.000002,0.000037,0.000039,0.000030,0.000026,-0.000016,-0.000014,0.000004,0.000018,...,-0.000021,-0.000008,-0.000035,-0.000045,-0.000066,-0.000039,-0.000033,-0.000048,-0.000039,-0.000039
1,-0.000012,-0.000024,0.000001,-0.000002,-0.000015,-0.000022,-0.000055,-0.000036,-0.000027,-0.000025,...,-0.000050,-0.000040,-0.000068,-0.000065,-0.000084,-0.000052,-0.000021,-0.000042,-0.000031,-0.000034
2,-0.000077,-0.000078,-0.000059,-0.000065,-0.000063,-0.000055,-0.000067,-0.000088,-0.000071,-0.000065,...,-0.000017,-0.000022,-0.000050,-0.000035,-0.000048,-0.000018,-0.000020,-0.000042,-0.000029,-0.000027
3,-0.000066,-0.000067,-0.000050,-0.000065,-0.000060,-0.000055,-0.000068,-0.000062,-0.000053,-0.000054,...,-0.000039,-0.000060,-0.000078,-0.000064,-0.000068,-0.000041,-0.000044,-0.000062,-0.000034,-0.000043
4,-0.000045,-0.000055,-0.000033,-0.000053,-0.000054,-0.000063,-0.000083,-0.000052,-0.000050,-0.000053,...,-0.000044,-0.000055,-0.000070,-0.000054,-0.000063,-0.000037,-0.000060,-0.000070,-0.000034,-0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307995,0.000077,0.000051,0.000020,-0.000052,-0.000064,0.000143,-0.000131,0.000119,-0.000013,-0.000012,...,0.000050,-0.000005,-0.000011,-0.000044,0.000001,0.000042,0.000003,0.000048,-0.000040,-0.000022
1307996,0.000071,0.000063,0.000092,-0.000048,-0.000063,0.000181,-0.000130,0.000136,-0.000006,0.000016,...,0.000052,0.000007,0.000006,-0.000023,0.000016,0.000049,0.000020,0.000069,-0.000026,-0.000004
1307997,0.000062,0.000065,0.000119,-0.000045,-0.000058,0.000166,-0.000122,0.000145,-0.000007,0.000020,...,0.000042,-0.000002,0.000001,-0.000022,0.000010,0.000039,0.000013,0.000067,-0.000029,-0.000012
1307998,0.000052,0.000061,0.000111,-0.000049,-0.000056,0.000152,-0.000108,0.000123,-0.000017,0.000011,...,0.000038,-0.000011,-0.000006,-0.000029,0.000004,0.000036,0.000008,0.000057,-0.000035,-0.000014


In [22]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [23]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [24]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
32119/40875 [======================>.......] - ETA: 10s - loss: 2.3834

40875/40875 [==============================] - 55s 1ms/step - loss: 2.2650 - val_loss: 2.9046
Epoch 2/10
40875/40875 [==============================] - 54s 1ms/step - loss: 1.6664 - val_loss: 2.9964
Epoch 3/10
40875/40875 [==============================] - 54s 1ms/step - loss: 1.5095 - val_loss: 2.9952
Epoch 4/10
40875/40875 [==============================] - 54s 1ms/step - loss: 1.4317 - val_loss: 3.1108
Epoch 5/10
40875/40875 [==============================] - 54s 1ms/step - loss: 1.3864 - val_loss: 3.0446
Epoch 6/10
40875/40875 [==============================] - 54s 1ms/step - loss: 1.3539 - val_loss: 3.0394
Epoch 7/10
40875/40875 [==============================] - 54s 1ms/step - loss: 1.3219 - val_loss: 3.1125
Epoch 8/10
40875/40875 [==============================] - 54s 1ms/step - loss: 1.2889 - val_loss: 3.0289
Epoch 9/10
40875/40875 [==============================] - 54s 1ms/step - loss: 1.2662 - val_loss: 3.1235
Epoch 10/10
40875/40875 [==============================] - 54s 1ms

In [25]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

5106/5106 [==============================] - 4s 795us/step
              precision    recall  f1-score   support

           0       0.23      0.34      0.28      1499
           1       0.27      0.31      0.29      1499
           2       0.18      0.32      0.23      1499
           3       0.47      0.26      0.34      1499
           4       0.82      0.81      0.81      1499
           5       0.30      0.44      0.35      1499
           6       0.60      0.52      0.56      1499
           7       0.76      0.36      0.49      1499
           8       0.29      0.21      0.24      1499
           9       0.37      0.53      0.44      1499
          10       0.29      0.25      0.27      1499
          11       0.28      0.36      0.32      1499
          12       0.32      0.12      0.17      1499
          13       0.18      0.34      0.24      1499
          14       0.17      0.11      0.13      1499
          15       0.56      0.36      0.44      1499
          16       0.2

## 0-32

In [26]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [27]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_28746/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_28746/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_28746/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [28]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

: 

In [ ]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [ ]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
 1816/40875 [>.............................] - ETA: 1:08 - loss: 3.6163

40875/40875 [==============================] - 73s 2ms/step - loss: 1.6391 - val_loss: 2.2969
Epoch 2/10
40875/40875 [==============================] - 72s 2ms/step - loss: 0.8422 - val_loss: 2.0763
Epoch 3/10
40875/40875 [==============================] - 71s 2ms/step - loss: 0.6961 - val_loss: 2.0895
Epoch 4/10
40875/40875 [==============================] - 77s 2ms/step - loss: 0.6233 - val_loss: 1.9373
Epoch 5/10
40875/40875 [==============================] - 72s 2ms/step - loss: 0.5804 - val_loss: 2.1325
Epoch 6/10
40875/40875 [==============================] - 72s 2ms/step - loss: 0.5495 - val_loss: 2.0038
Epoch 7/10
40875/40875 [==============================] - 71s 2ms/step - loss: 0.5240 - val_loss: 2.1145
Epoch 8/10
40875/40875 [==============================] - 71s 2ms/step - loss: 0.5012 - val_loss: 2.0442
Epoch 9/10
40875/40875 [==============================] - 78s 2ms/step - loss: 0.4831 - val_loss: 2.0025
Epoch 10/10
40875/40875 [==============================] - 74s 2ms

In [ ]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

5106/5106 [==============================] - 6s 1ms/step
              precision    recall  f1-score   support

           0       0.53      0.80      0.63      1499
           1       0.63      0.80      0.70      1499
           2       0.47      0.75      0.58      1499
           3       0.81      0.38      0.52      1499
           4       0.95      0.91      0.93      1499
           5       0.85      0.89      0.87      1499
           6       0.86      0.83      0.84      1499
           7       0.87      0.57      0.69      1499
           8       0.49      0.74      0.59      1499
           9       0.60      0.75      0.67      1499
          10       0.64      0.63      0.64      1499
          11       0.97      0.86      0.91      1499
          12       0.63      0.64      0.63      1499
          13       0.52      0.48      0.50      1499
          14       0.56      0.54      0.55      1499
          15       0.85      0.32      0.47      1499
          16       0.70 

## 0-64

In [ ]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [ ]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [ ]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

NameError: name 'tf' is not defined

In [ ]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

: 

In [ ]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

  60/5106 [..............................] - ETA: 4s  

5106/5106 [==============================] - 5s 919us/step
              precision    recall  f1-score   support

           0       0.75      0.95      0.84      1499
           1       0.89      0.89      0.89      1499
           2       0.65      0.63      0.64      1499
           3       0.93      0.97      0.95      1499
           4       0.98      1.00      0.99      1499
           5       0.95      0.97      0.96      1499
           6       0.95      0.95      0.95      1499
           7       0.97      0.60      0.74      1499
           8       0.60      0.60      0.60      1499
           9       0.47      0.84      0.60      1499
          10       0.83      0.91      0.86      1499
          11       0.96      0.91      0.93      1499
          12       0.80      0.76      0.78      1499
          13       0.83      0.81      0.82      1499
          14       0.76      0.68      0.72      1499
          15       0.98      0.41      0.58      1499
          16       0.6

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators = 500, learning_rate = 0.5,verbosity = 1, random_state = 55)
xgb_model.fit(x,y, eval_set = [(xv,yv)], early_stopping_rounds = 10)
y_pred=xgb_model.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:4.43284
[1]	validation_0-mlogloss:4.35325
[2]	validation_0-mlogloss:4.20869
[3]	validation_0-mlogloss:4.06689
[4]	validation_0-mlogloss:3.88122
[5]	validation_0-mlogloss:3.77287
[6]	validation_0-mlogloss:3.69161
[7]	validation_0-mlogloss:3.57765
[8]	validation_0-mlogloss:3.51041
[9]	validation_0-mlogloss:3.43149
[10]	validation_0-mlogloss:3.38192
[11]	validation_0-mlogloss:3.32379
[12]	validation_0-mlogloss:3.28331
[13]	validation_0-mlogloss:3.21093
[14]	validation_0-mlogloss:3.18555
[15]	validation_0-mlogloss:3.12841
[16]	validation_0-mlogloss:3.09847
[17]	validation_0-mlogloss:3.05379
[18]	validation_0-mlogloss:3.01431
[19]	validation_0-mlogloss:2.98414
[20]	validation_0-mlogloss:2.95681
[21]	validation_0-mlogloss:2.93154
[22]	validation_0-mlogloss:2.91297
[23]	validation_0-mlogloss:2.88990
[24]	validation_0-mlogloss:2.86761
[25]	validation_0-mlogloss:2.84620
[26]	validation_0-mlogloss:2.82552
[27]	validation_0-mlogloss:2.79681
[28]	validation_0-mlogloss:2.7